In [ ]:
import brainlit
from brainlit.utils.session import NeuroglancerSession
import matplotlib.pyplot as plt
import numpy as np
import napari
from scipy import ndimage
from pathlib import Path
import skimage
import pytest
import glob
%gui qt

brainlit_path=Path.cwd()
ImgPath=Path.joinpath(brainlit_path,'ffn')
print(ImgPath)
filelist = glob.glob('ImgPath\*.png')
print(filelist)
# img = np.load(filelist)
# print('image shape =',img.shape)
# napari.view_image(img)


## Collect locations of known somas

In [ ]:
for seg_id in range(1000):
    try:
        vox = ngl_sess._get_voxel(seg_id, v_id) 
        voxs.append(vox)
        _,vox2,_ = ngl_sess.pull_voxel(seg_id, v_id)
        voxs2.append(np.array(np.floor(vox2.center()[:])))
    except:
        pass

In [ ]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"
mip = 0; radius = 150; v_id = 0
# voxs = []
voxs2 = []
ngl_sess = NeuroglancerSession(mip=mip, url=dir, url_segments=dir_segments)
# for seg_id in range(1000):
#     try:
#         vox = ngl_sess._get_voxel(seg_id, v_id) 
#         voxs.append(vox)
#     except:
#         pass

for seg_id in range(1000):
    try:
        _,vox2,_ = ngl_sess.pull_voxel(seg_id, v_id)
        voxs2.append(np.array(np.floor(vox2.center()[:])))
    except:
        pass

# np.testing.assert_array_equal(voxs,voxs2)

In [ ]:
len(voxs2)

In [ ]:

Nbbox = [956341.1875,2522349.25,6056794.5,1056341.1875,2622349.25,6156794.5]
KnoSomNum = 0
for vox in voxs2:
    if vox[0] > Nbbox[0] and vox[0] < Nbbox[3]:
        if vox[1] > Nbbox[1] and vox[1] < Nbbox[4]:
            if vox[2] > Nbbox[2] and vox[2] < Nbbox[5]:
                KnoSomNum +=1

print('Known number of soma within the subvolume is',str(KnoSomNum))


## Simple Soma Detector
### threshold top 5% followed by binary erosion

In [ ]:
%gui qt
### threshold top 5%
H = np.sort(np.reshape(img,[1,-1]))
thres = H[0,-round((H.shape[1])*0.05)]
lx,ly,lz = img.shape
imgTop5 = np.zeros([lx,ly,lz])
for m in range(lx):
    for n in range(ly):
        for p in range(lz):
            if img[m,n,p] < thres:
                imgTop5[m,n,p] = 0
            else:
                imgTop5[m,n,p] = img[m,n,p]

# viewer = napari.view_image(img, name='Original Image', colormap='red')
# viewer.add_labels(imgTop5, name='Gray Threshold at 5%', num_colors=2, opacity=0.3)

### binary erosion
ero5 = ndimage.binary_erosion(imgTop5, iterations=5)

# viewer = napari.view_image(img, name='Original Image', colormap='red')
# viewer.add_labels(ero5, name='Gray Erosion', num_colors=2, opacity=0.5)

### region label
ReLab, NumLab = skimage.measure.label(ero5, return_num=True)
print('number of labels = ',NumLab)

viewer = napari.view_image(img, name='Gray Erosion', colormap='green')
viewer.add_labels(ReLab, name='Region Labels', opacity=0.5)
# num_colors=20
props = skimage.measure.regionprops(ReLab)
CNumLab = NumLab
for x in range(NumLab):
    D = props[x].equivalent_diameter
    #print(D)
    if D < 10:
        CNumLab-=1

print('Possible number of somas is',str(CNumLab))


### Other methods

In [ ]:
## Binarize at top 0.1%
H = np.sort(np.reshape(img,[1,-1]))
thres = H[0,-round((H.shape[1])*0.001)]
lx,ly,lz = img.shape
imgBW = np.zeros([lx,ly,lz])
for m in range(lx):
    for n in range(ly):
        for p in range(lz):
            if img[m,n,p] < thres:
                imgBW[m,n,p] = 0
            else:
                imgBW[m,n,p] = 1

# viewer = napari.view_image(img, name='Original Image', colormap='red')
# viewer.add_labels(imgBW, name='Binary Threshold at 5%', num_colors=2, opacity=0.5)
## gray erosion
Gero = ndimage.grey_erosion(imgTop5, size=2)

viewer = napari.view_image(img, name='Original Image', colormap='red')
viewer.add_labels(Gero, name='Gray Erosion', num_colors=2, opacity=0.5)
